In [52]:
from transformers import BertForMaskedLM, BertTokenizer
import gradio as gr
import torch

In [53]:
model = BertForMaskedLM.from_pretrained('/data/teamwork/multimodal/pretrained_models/chinese-roberta-gov-affairs-wwm')
tokenizer = BertTokenizer.from_pretrained('/data/teamwork/multimodal/pretrained_models/chinese-roberta-gov-affairs-wwm')

Some weights of BertForMaskedLM were not initialized from the model checkpoint at /data/teamwork/multimodal/pretrained_models/chinese-roberta-gov-affairs-wwm and are newly initialized: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [54]:
device = 'cuda:7' if torch.cuda.is_available() else 'cpu'

In [55]:
model = model.to(device)
model.eval()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [56]:
import re

def fn(text: str):
    inferred_token = [text]

    # gradually fill in the MASK tokens, one by one
    # based on demo.py
    with torch.no_grad():
        inputs = tokenizer(inferred_token, return_tensors="pt").to(device)
        logits = model(**inputs).logits
    mask_token_index = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]
    predicted_token_id = logits[0, mask_token_index].argmax(axis=-1)
    return tokenizer.decode(predicted_token_id)

In [19]:
demo = gr.Interface(fn, inputs=['text'], outputs=['text'])

In [20]:
demo.launch(share=True, server_port=6006)

Running on local URL:  http://127.0.0.1:7860/

To create a public link, set `share=True` in `launch()`.


(<gradio.routes.App at 0x7f544566f220>, 'http://127.0.0.1:7860/', None)